In [1]:
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time
from astropy import units as u
import datetime
import urllib.parse
import requests
import os
from PIL import Image
import matplotlib.pyplot as plt

# Step 1: Define Hamburg location and current time
hamburg = EarthLocation(lat=53.5511*u.deg, lon=9.9937*u.deg, height=0*u.m)
now = Time(datetime.datetime.utcnow())

# Step 2: Get zenith (directly overhead) sky coordinates
zenith = AltAz(alt=90*u.deg, az=0*u.deg, location=hamburg, obstime=now)
sky_coord = zenith.transform_to('icrs')
ra_str = f"{sky_coord.ra.deg:.6f}"
dec_str = f"{sky_coord.dec.deg:.6f}"

# Step 3: Query SkyView using selected surveys (choose a few for demo)
surveys = ["DSS", "2MASS-K", "GALEXGR6-AIS", "WISE 3.4"]  # Can add more
output_dir = "sky_images"
os.makedirs(output_dir, exist_ok=True)

# Step 4: Fetch, save, and display each image
for survey in surveys:
    params = {
        "Position": f"{ra_str}, {dec_str}",
        "Survey": survey,
        "Coordinates": "J2000",
        "Return": "PNG",
        "Sampler": "Nearest",
        "Scaling": "Linear",
        "Pixels": "500",
        "Size": "2.0"
    }
    url = f"https://skyview.gsfc.nasa.gov/current/cgi/runquery.pl?{urllib.parse.urlencode(params)}"
    print(f"Fetching image from survey: {survey}")
    
    # Get the SkyView response page
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to query SkyView for {survey}")
        continue

    # Extract the PNG image URL from the HTML response
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    image_url = None
    for link in links:
        if link['href'].lower().endswith(".png"):
            image_url = link['href']
            break

    if not image_url:
        print(f"No image URL found for {survey}")
        continue

    full_image_url = urllib.parse.urljoin(url, image_url)
    image_data = requests.get(full_image_url).content

    filename = os.path.join(output_dir, f"{survey.replace(' ', '_')}.png")
    with open(filename, 'wb') as f:
        f.write(image_data)
    print(f"Saved: {filename}")

    # Display the image
    img = Image.open(filename)
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(f"{survey} (RA: {ra_str}, Dec: {dec_str})")
    plt.axis('off')
    plt.show()


C:\Users\rforner\AppData\Local\Temp\ipykernel_17548\3180248593.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = Time(datetime.datetime.utcnow())


ConvertError: Cannot transform from <class 'astropy.coordinates.builtin_frames.altaz.AltAz'> to <class 'str'>

In [ ]:
from astropy.coordinates import EarthLocation, AltAz, ICRS
from astropy.time import Time
from astropy import units as u
import datetime
import urllib.parse
import requests
import os
from PIL import Image
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

# Step 1: Define Hamburg location and current UTC time
hamburg = EarthLocation(lat=53.5511*u.deg, lon=9.9937*u.deg, height=0*u.m)
now = Time(datetime.datetime.utcnow())

# Step 2: Compute zenith (alt=90°, directly overhead) in ICRS (RA/Dec)
zenith_altaz = AltAz(alt=90*u.deg, az=0*u.deg, location=hamburg, obstime=now)
sky_coord = zenith_altaz.transform_to(ICRS())
ra_str = f"{sky_coord.ra.deg:.6f}"
dec_str = f"{sky_coord.dec.deg:.6f}"

print(f"Zenith at Hamburg (RA, Dec): {ra_str}, {dec_str}")

# Step 3: Choose surveys (you can add more)
surveys = ["DSS", "2MASS-K", "GALEXGR6-AIS", "WISE 3.4"]
output_dir = "sky_images"
os.makedirs(output_dir, exist_ok=True)

# Step 4: Fetch, save, and display images
for survey in surveys:
    params = {
        "Position": f"{ra_str}, {dec_str}",
        "Survey": survey,
        "Coordinates": "J2000",
        "Return": "PNG",
        "Sampler": "Nearest",
        "Scaling": "Linear",
        "Pixels": "500",
        "Size": "2.0"
    }
    url = f"https://skyview.gsfc.nasa.gov/current/cgi/runquery.pl?{urllib.parse.urlencode(params)}"
    print(f"\nFetching image from survey: {survey}")

    # Get SkyView response HTML
    response = requests.get(url)
    if response.status_code != 200:
        print(f"❌ Failed to get response from SkyView for {survey}")
        continue

    # Parse HTML to find image URL
    soup = BeautifulSoup(response.text, 'html.parser')
    image_url = None
    for link in soup.find_all('a'):
        if link.get('href', '').lower().endswith(".png"):
            image_url = link['href']
            break

    if not image_url:
        print(f"⚠️ No image found for survey: {survey}")
        continue

    # Download and save the image
    full_image_url = urllib.parse.urljoin(url, image_url)
    image_data = requests.get(full_image_url).content
    filename = os.path.join(output_dir, f"{survey.replace(' ', '_')}.png")
    with open(filename, 'wb') as f:
        f.write(image_data)
    print(f"✅ Saved image: {filename}")

    # Display image using matplotlib
    img = Image.open(filename)
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(f"{survey} (RA: {ra_str}, Dec: {dec_str})")
    plt.axis('off')
    plt.show()


C:\Users\rforner\AppData\Local\Temp\ipykernel_19772\3792465725.py:14: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = Time(datetime.datetime.utcnow())


Zenith at Hamburg (RA, Dec): 135.933198, 53.647799

Fetching image from survey: DSS
⚠️ No image found for survey: DSS

Fetching image from survey: 2MASS-K
⚠️ No image found for survey: 2MASS-K

Fetching image from survey: GALEXGR6-AIS
⚠️ No image found for survey: GALEXGR6-AIS

Fetching image from survey: WISE 3.4
⚠️ No image found for survey: WISE 3.4


In [ ]:
from astropy.coordinates import EarthLocation, AltAz, ICRS
from astropy.time import Time
from astropy import units as u
import datetime
import urllib.parse
import requests
import os
from PIL import Image
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup


# Step 1: Define Hamburg location and current UTC time
hamburg = EarthLocation(lat=53.5511*u.deg, lon=9.9937*u.deg, height=0*u.m)
now = Time(datetime.datetime.utcnow())

# Step 2: Compute zenith (alt=90°) in ICRS (RA/Dec)
zenith_altaz = AltAz(alt=90*u.deg, az=0*u.deg, location=hamburg, obstime=now)
sky_coord = zenith_altaz.transform_to(ICRS())
ra_str = f"{sky_coord.ra.deg:.6f}"
dec_str = f"{sky_coord.dec.deg:.6f}"

print(f"Zenith at Hamburg (RA, Dec): {ra_str}, {dec_str}")

# Step 3: Fetch and loop through all surveys
surveys = [
    "DSS", "DSS2 Red", "DSS2 Blue", "DSS2 IR",
    "2MASS-K", "2MASS-H", "2MASS-J",
    "WISE 3.4", "WISE 4.6", "WISE 12", "WISE 22",
    "GALEXGR6-AIS", "GALEXGR6-MIS",
    "SDSSg", "SDSSr", "SDSSi", "SDSSz",
    "XMM-EPIC", "ROSAT All Sky Survey", "IRAS 100", "IRAS 60"
]
output_dir = "sky_images"
os.makedirs(output_dir, exist_ok=True)

# Optional: limit the number for quick testing
# surveys = surveys[:10]

for survey in surveys:
    params = {
        "Position": f"{ra_str}, {dec_str}",
        "Survey": survey,
        "Coordinates": "J2000",
        "Return": "PNG",
        "Sampler": "Nearest",
        "Scaling": "Linear",
        "Pixels": "500",
        "Size": "2.0"
    }
    url = f"https://skyview.gsfc.nasa.gov/current/cgi/runquery.pl?{urllib.parse.urlencode(params)}"
    print(f"\nFetching image from survey: {survey}")

    try:
        response = requests.get(url)
        print(response.status_code, response.reason, response.text)
        if response.status_code != 200:
            print(f"❌ Failed: {survey}")
            
            continue


        soup = BeautifulSoup(response.text, 'html.parser')
        image_url = None
        for link in soup.find_all('a'):
            if link.get('href', '').lower().endswith(".png"):
                image_url = link['href']
                break

        if not image_url:
            print(f"⚠️ No image found for: {survey}")
            continue

        full_image_url = urllib.parse.urljoin(url, image_url)
        image_data = requests.get(full_image_url).content
        filename = os.path.join(output_dir, f"{survey.replace(' ', '_')}.png")
        with open(filename, 'wb') as f:
            f.write(image_data)
        print(f"✅ Saved: {filename}")

        # Optional: comment out display for speed
        img = Image.open(filename)
        plt.figure(figsize=(5, 5))
        plt.imshow(img)
        plt.title(survey)
        plt.axis('off')
        plt.show()

    except Exception as e:
        print(f"⚠️ Error fetching {survey}: {e}")


C:\Users\rforner\AppData\Local\Temp\ipykernel_17548\2709601858.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = Time(datetime.datetime.utcnow())


Zenith at Hamburg (RA, Dec): 59.858813, 53.479728

Fetching image from survey: DSS
200 OK <html>
  <head>
    <link rel="stylesheet" href="../includes/skyviewall.css">

    <title>SkyView images</title>
    <script language='JavaScript' src='../javascript/deproj.js'  ></script>
    <script language='JavaScript' src='../javascript/astro.js'   ></script>
    <script language='JavaScript' src='../javascript/scaler.js'> </script>
    <script language='JavaScript' src='../javascript/img.js'     ></script>

<!--
This content is required for inclusion in the NASA Google Analytics
framework.  This file will be included within the <HEAD> element of all
displayed SkyView web pages.
  Copied from includes/googleanalytics.html
-->
<script 
  id="fed_an_js_tag" 
  type="text/javascript"
  src="https://asd.gsfc.nasa.gov/scripts/Federated-Analytics.js?agency=NASA"
>
<!-- Google Analytics -->
</script>
  </head>
  <body bgcolor="#ffffff">
    <MAP NAME="buttonbar" id="buttonbar">
<AREA alt="SkyView Ho